<a href="https://colab.research.google.com/github/ashbornarise/Machine-Learning-research/blob/main/Prediction_Bras_Robotique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [33]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),    # toutes les images auront la même taille
    transforms.ToTensor(),          # conversion en tenseur PyTorch (canal, hauteur, largeur)
])


In [34]:
dataset_path = "dataset/"  # le dossier que tu as préparé
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [35]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [36]:
for images, labels in train_loader:
    print("Batch d'images :", images.shape)   # (batch_size, 3, 64, 64)
    print("Étiquettes :", labels)
    break


Batch d'images : torch.Size([8, 3, 64, 64])
Étiquettes : tensor([1, 2, 2, 0, 2, 2, 1, 2])


In [37]:
import torch
import torch.nn as nn

In [38]:
class MonCNN(nn.Module):
    def __init__(self, num_classes):
        super(MonCNN, self).__init__()

        # Couche convolutionnelle
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # (3 canaux RGB → 16 filtres)
            nn.ReLU(),
            nn.MaxPool2d(2),  # Réduction de taille de moitié

            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (16 → 32)
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Couche entièrement connectée
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 64),  # 64 neurones
            nn.ReLU(),
            nn.Linear(64, num_classes)   # Sortie finale = nombre de classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [39]:
model = MonCNN(num_classes=len(train_dataset.classes))

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MonCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [42]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 1. Remise à zéro du gradient
        optimizer.zero_grad()

        # 2. Passage avant (prédiction)
        outputs = model(images)

        # 3. Calcul de la perte
        loss = criterion(outputs, labels)

        # 4. Rétropropagation
        loss.backward()

        # 5. Mise à jour des poids
        optimizer.step()

        # Statistiques
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"🔁 Époque {epoch+1}/{epochs} - Perte : {running_loss:.3f} - Précision : {100 * correct / total:.2f}%")

🔁 Époque 1/5 - Perte : 4.627 - Précision : 24.00%
🔁 Époque 2/5 - Perte : 4.505 - Précision : 40.00%
🔁 Époque 3/5 - Perte : 4.196 - Précision : 56.00%
🔁 Époque 4/5 - Perte : 4.096 - Précision : 44.00%
🔁 Époque 5/5 - Perte : 4.180 - Précision : 60.00%


In [43]:
torch.save(model.state_dict(), "modele_cnn.pth")
from google.colab import files
files.download("modele_cnn.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>